### Necessary Imports 

In [2]:
import numpy as np
import tensorflow as tf

In [26]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model


### Loading Dataset 

In [44]:
# load MNIST

(X_train, y_train), (X_test, y_test) = mnist.load_data()

image_size = X_train.shape[1]
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)
X_train.astype('float32')/255.
X_test.astype('float32')/255.


# number of classes
num_labels = len(np.unique(y_train))
print(num_labels)

# One Hot Encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



10


### Network Parameters 

In [5]:
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
dropout = 0.2
n_filters = 32



### Model Architecture ( Left Branch) 

In [50]:
left_inputs = Input(shape=input_shape)
x = left_inputs
filters = n_filters
for i in range(3):
    x = Conv2D(filters=filters,
              kernel_size=kernel_size,
              padding='same',
              activation='relu')(x)
    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)
    filters *= 2
    

### Model Architecture (Right  Branch) 

In [51]:
right_inputs = Input(shape=input_shape)
filters = n_filters
y = right_inputs
for i in range(3):
    y = Conv2D(filters=filters,
              kernel_size=kernel_size,
              padding='same',
              activation='relu')(y)
    
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)
    filters *= 2
    

### Concatenation 

In [52]:
y = concatenate([x,y])
y = Flatten()(y)
y = Dropout(dropout)(y)
y = Dense(128, activation='relu')(y)
outputs = Dense(num_labels, activation='softmax')(y)


model = Model([left_inputs, right_inputs], outputs)
plot_model( model, to_file='model.png', show_shapes=False, show_layer_names=True,
           rankdir='TB', expand_nested=False, dpi=96)
model.summary()

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 28, 28, 32)   320         input_15[0][0]                   
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 28, 28, 32)   320         input_16[0][0]                   
_____

### Model Compile and Train 

In [53]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit([X_train, X_train],
          y_train,
          validation_data=([X_test, X_test], y_test),
          epochs=3,
          batch_size=batch_size,
          verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 168s 3ms/sample - loss: 0.4110 - accuracy: 0.9374 - val_loss: 0.0667 - val_accuracy: 0.9793
Epoch 2/3
60000/60000 [==============================] - 174s 3ms/sample - loss: 0.0835 - accuracy: 0.9748 - val_loss: 0.0364 - val_accuracy: 0.9883
Epoch 3/3
60000/60000 [==============================] - 175s 3ms/sample - loss: 0.0730 - accuracy: 0.9785 - val_loss: 0.0484 - val_accuracy: 0.9875
